In [3]:
import numpy as np
from astropy import units as u
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
import matplotlib.pyplot as plt
import random
import plotly.graph_objects as go
import math
import csv
import os

In [4]:
# space parameter difinition

mu_earth = 398600.4418



class ExtendedOrbit(Orbit):
    
    def __init__(self, *args, separation_sec=None, orbit_type=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.noeuds = []
        self.nb_noeuds = 0
        self.separation_sec = separation_sec
        self.ttype = orbit_type

    def add_noeud(self, noeud):
        self.noeuds.append(noeud)
        self.nb_noeuds += 1


class Nodes:
    
    def __init__(self, ID, orbit, true_anomaly):
        self.id = ID
        self.orbit = orbit
        self.true_anomaly = true_anomaly


class Arc:
    
    def __init__(self, node_i, node_j, tau_ij, tau_ij_steptime, phi_ij, psi_ij, maneuver_type):
        self.node_i = node_i
        self.node_j = node_j
        self.tau_ij = tau_ij
        self.tau_ij_steptime = tau_ij_steptime
        self.phi_ij = phi_ij
        self.psi_ij = psi_ij
        self.maneuver_type = maneuver_type


class Task:
    
    def __init__(self, ID, weight, starting_orbit, duration = 0):
        self.id = ID
        self.weight = weight
        self.starting_orbit = starting_orbit
        self.starting_node = None  # Will be set randomly
        self.duration = duration
        self.sub_tasks = []


class refueling_depot:

    def __init__(self, ID, starting_orbit):
        self.id = ID
        self.starting_orbit = starting_orbit
        self.starting_node = None  # Will be set randomly


class SubTask:
    
    def __init__(self, node, time, task):
        self.node = node
        self.time = time
        self.task = task
        self.completed = 0  # 0: to complete, 1: completed


class Robot_serviser:

    def __init__(self, ID, max_fuel, current_orbit, current_node, R_Fd):
        self.id = ID
        self.max_fuel = max_fuel
        self.current_orbit = current_orbit
        self.current_node = current_node 
        self.R_Fd = R_Fd


def angle_difference(angle1, angle2):
    if ((angle2 - angle1 + 180) % 360) == 0:
        diff = (angle2 - angle1)
    else: 
        diff = (angle2 - angle1 + 180) % 360 - 180
    return diff

In [20]:
# defintion of the environnement 

time_horizon = 2_592_000
step_time = time_horizon / 1440
time_horizon_steptime = int(time_horizon / step_time)

P1 = 5400
P2 = 5400
P3 = 5400
P4 = 43200
P5 = 43200
P6 = 43200
P7 = 43200
P8 = 86400

P1_test = 6000
P2_test = 8000
P3_test = 10000



# orbit_test 1 definition 
a_o1_test = (((mu_earth * P1_test**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o1_test = 0 * u.one          # Excentricité (dimensionless)
inc_o1_test = 0 * u.deg     # Inclinaison
raan_o1_test = 0 * u.deg   # Ascension droite du nœud ascendant
argp_o1_test = 0 * u.deg         # Argument du périgée
nu_o1_test = 0 * u.deg           # Anomalie vraie


# orbit_test 2 definition 
a_o2_test = (((mu_earth * P2_test**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o2_test = 0 * u.one          # Excentricité (dimensionless)
inc_o2_test = 0 * u.deg     # Inclinaison
raan_o2_test = 0 * u.deg   # Ascension droite du nœud ascendant
argp_o2_test = 0 * u.deg         # Argument du périgée
nu_o2_test = 0 * u.deg           # Anomalie vraie


# orbit_test 3 definition 
a_o3_test = (((mu_earth * P3_test**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o3_test = 0 * u.one          # Excentricité (dimensionless)
inc_o3_test = 52.986 * u.deg     # Inclinaison
raan_o3_test = 290.269 * u.deg   # Ascension droite du nœud ascendant
argp_o3_test = 0 * u.deg         # Argument du périgée
nu_o3_test = 0 * u.deg           # Anomalie vraie



# orbit 1 definition 
a_o1 = (((mu_earth * P1**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o1 = 0 * u.one          # Excentricité (dimensionless)
inc_o1 = 52.986 * u.deg     # Inclinaison
raan_o1 = 290.269 * u.deg   # Ascension droite du nœud ascendant
argp_o1 = 0 * u.deg         # Argument du périgée
nu_o1 = 0 * u.deg           # Anomalie vraie


# orbit 2 definition
a_o2 = (((mu_earth * P2**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe            
ecc_o2 = 0 * u.one          # Excentricité (dimensionless)
inc_o2 = 52.989 * u.deg     # Inclinaison
raan_o2 = 120.183 * u.deg   # Ascension droite du nœud ascendant
argp_o2 = 0 * u.deg         # Argument du périgée
nu_o2 = 0 * u.deg           # Anomalie vraie


# orbit 3 definition
a_o3 = (((mu_earth * P3**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o3 = 0 * u.one          # Excentricité (dimensionless)
inc_o3 = 53.000 * u.deg     # Inclinaison
raan_o3 = 64.290 * u.deg    # Ascension droite du nœud ascendant
argp_o3 = 0 * u.deg         # Argument du périgée
nu_o3 = 0 * u.deg           # Anomalie vraie


# orbit 4 definition
a_o4 = (((mu_earth * P4**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o4 = 0 * u.one         # Excentricité (dimensionless)
inc_o4 = 0.000 * u.deg     # Inclinaison
raan_o4 = 0.000 * u.deg    # Ascension droite du nœud ascendant
argp_o4 = 0 * u.deg        # Argument du périgée
nu_o4 = 0 * u.deg          # Anomalie vraie


# orbit 5 definition
a_o5 = (((mu_earth * P5**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o5 = 0 * u.one         # Excentricité (dimensionless)
inc_o5 = 55.000 * u.deg     # Inclinaison
raan_o5 = 300.000 * u.deg    # Ascension droite du nœud ascendant
argp_o5 = 0 * u.deg        # Argument du périgée
nu_o5 = 0 * u.deg          # Anomalie vraie


# orbit 6 definition
a_o6 = (((mu_earth * P6**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o6 = 0 * u.one         # Excentricité (dimensionless)
inc_o6 = 55.000 * u.deg     # Inclinaison
raan_o6 = 45.000 * u.deg    # Ascension droite du nœud ascendant
argp_o6 = 0 * u.deg        # Argument du périgée
nu_o6 = 0 * u.deg          # Anomalie vraie


# orbit 7 definition
a_o7 = (((mu_earth * P7**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o7 = 0 * u.one         # Excentricité (dimensionless)
inc_o7 = 55.000 * u.deg     # Inclinaison
raan_o7 = 225.000 * u.deg    # Ascension droite du nœud ascendant
argp_o7 = 0 * u.deg        # Argument du périgée
nu_o7 = 0 * u.deg          # Anomalie vraie


# orbit 8 definition
a_o8 = (((mu_earth * P8**2) / (4 * np.pi ** 2))**(1/3)) * u.km    # Demi-grand axe           
ecc_o8 = 0 * u.one         # Excentricité (dimensionless)
inc_o8 = 0.000 * u.deg     # Inclinaison
raan_o8 = 0.000 * u.deg    # Ascension droite du nœud ascendant
argp_o8 = 0 * u.deg        # Argument du périgée
nu_o8 = 0 * u.deg          # Anomalie vraie



# creation of the orbits
orbit1_test = ExtendedOrbit.from_classical(Earth, a_o1_test, ecc_o1_test, inc_o1_test, raan_o1_test, argp_o1_test, nu_o1_test)
orbit2_test = ExtendedOrbit.from_classical(Earth, a_o2_test, ecc_o2_test, inc_o2_test, raan_o2_test, argp_o2_test, nu_o2_test)
orbit3_test = ExtendedOrbit.from_classical(Earth, a_o3_test, ecc_o3_test, inc_o3_test, raan_o3_test, argp_o3_test, nu_o3_test)

orbit1 = ExtendedOrbit.from_classical(Earth, a_o1, ecc_o1, inc_o1, raan_o1, argp_o1, nu_o1)
orbit2 = ExtendedOrbit.from_classical(Earth, a_o2, ecc_o2, inc_o2, raan_o2, argp_o2, nu_o2)
orbit3 = ExtendedOrbit.from_classical(Earth, a_o3, ecc_o3, inc_o3, raan_o3, argp_o3, nu_o3)
orbit4 = ExtendedOrbit.from_classical(Earth, a_o4, ecc_o4, inc_o4, raan_o4, argp_o4, nu_o4)
orbit5 = ExtendedOrbit.from_classical(Earth, a_o5, ecc_o5, inc_o5, raan_o5, argp_o5, nu_o5)
orbit6 = ExtendedOrbit.from_classical(Earth, a_o6, ecc_o6, inc_o6, raan_o6, argp_o6, nu_o6)
orbit7 = ExtendedOrbit.from_classical(Earth, a_o7, ecc_o7, inc_o7, raan_o7, argp_o7, nu_o7)
orbit8 = ExtendedOrbit.from_classical(Earth, a_o8, ecc_o8, inc_o8, raan_o8, argp_o8, nu_o8)



# Definition of the separtion parameter 
orbit1_test.separation_sec = 2000
orbit2_test.separation_sec = 2000
orbit3_test.separation_sec = 2000

orbit1.separation_sec = 1800
orbit2.separation_sec = 1800
orbit3.separation_sec = 1800
orbit4.separation_sec = 3600
orbit5.separation_sec = 3600
orbit6.separation_sec = 3600
orbit7.separation_sec = 3600
orbit8.separation_sec = 3600



# Definition of the orbit type
orbit1_test.ttype = "Leo"
orbit2_test.ttype = "Leo"
orbit3_test.ttype = "Leo"

orbit1.ttype = "Leo"
orbit2.ttype = "Leo"
orbit3.ttype = "Leo"
orbit4.ttype = "Meo"
orbit5.ttype = "Meo"
orbit6.ttype = "Meo"
orbit7.ttype = "Meo"
orbit8.ttype = "Geo"


# list to store all the orbit
O = []

#O.append(orbit1_test)
#O.append(orbit2_test)
#O.append(orbit3_test)

O.append(orbit1)
O.append(orbit2)
O.append(orbit3)
O.append(orbit4)
O.append(orbit5)
O.append(orbit6)
O.append(orbit7)
O.append(orbit8)

print(len(O))
print(step_time)






8
1800.0


In [21]:
def nodes_creation(O):
    N = []
    compt = 0
    for orbit in O:
        orbit.noeuds = []
        orbit.nb_noeuds = 0
        separation = orbit.separation_sec
        nb_nodes_per_orbit = int(round(orbit.period.value) / separation)
        for j in range(nb_nodes_per_orbit):
            true_anomaly = j * (360 * u.deg / nb_nodes_per_orbit)
            node = Nodes(compt, orbit, true_anomaly)
            N.append(node)
            orbit.add_noeud(node)
            compt += 1
    return N


def separationlist(O):
    separation_list = []
    for orbit in O:
        separation = 360 * u.deg / orbit.nb_noeuds
        #print (orbit.nb_noeuds)
        if separation not in separation_list:
            separation_list.append(separation)
    return separation_list

N = nodes_creation(O)
separation_list = separationlist(O)
print("Number of nodes: ", len(N))
print("Separation list: ", separation_list)


Number of nodes:  81
Separation list:  [<Quantity 120. deg>, <Quantity 30. deg>, <Quantity 15. deg>]


In [22]:
print_latex = True

if print_latex:
    compt = 1
    for orbit in O:
        print(f"${compt}^{orbit.ttype}$ & {orbit.noeuds[0].id},\dots,{orbit.noeuds[-1].id} & {round(orbit.a.value, 3)} & {round(orbit.inc.value, 3)} & {round(orbit.raan.value, 3)} & Starlink & {round(orbit.period.value)/60} & {orbit.separation_sec/60}/{360/((round(orbit.period.value)/60)/(orbit.separation_sec/60))}\\\ ")    




$1^Leo$ & 0,\dots,2 & 6652.556 & 52.986 & 290.269 & Starlink & 90.0 & 30.0/120.0\\ 
$1^Leo$ & 3,\dots,5 & 6652.556 & 52.989 & 120.183 & Starlink & 90.0 & 30.0/120.0\\ 
$1^Leo$ & 6,\dots,8 & 6652.556 & 53.0 & 64.29 & Starlink & 90.0 & 30.0/120.0\\ 
$1^Meo$ & 9,\dots,20 & 26610.223 & 0.0 & 0.0 & Starlink & 720.0 & 60.0/30.0\\ 
$1^Meo$ & 21,\dots,32 & 26610.223 & 55.0 & 300.0 & Starlink & 720.0 & 60.0/30.0\\ 
$1^Meo$ & 33,\dots,44 & 26610.223 & 55.0 & 45.0 & Starlink & 720.0 & 60.0/30.0\\ 
$1^Meo$ & 45,\dots,56 & 26610.223 & 55.0 & 225.0 & Starlink & 720.0 & 60.0/30.0\\ 
$1^Geo$ & 57,\dots,80 & 42241.096 & 0.0 & 0.0 & Starlink & 1440.0 & 60.0/15.0\\ 


In [23]:
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'grey']

fig = go.Figure()

# Plotting orbits and nodes with the same color
for i, orbit in enumerate(O):
    coords = orbit.sample()
    fig.add_trace(go.Scatter3d(x=coords.x.value, y=coords.y.value, z=coords.z.value,
                               mode='lines', name=f'Orbit {i+1} : {orbit.ttype}', line=dict(color=colors[i])))

    for node in orbit.noeuds:
        new_orbit = Orbit.from_classical(node.orbit.attractor, node.orbit.a, node.orbit.ecc, node.orbit.inc, node.orbit.raan, node.orbit.argp, node.true_anomaly)
        r = new_orbit.r  # Retrieve the position
        fig.add_trace(go.Scatter3d(x=[r[0].value], y=[r[1].value], z=[r[2].value],
                                   mode='markers', marker=dict(size=2, color=colors[i]), name=f"Node {node.id}, Anomaly {node.true_anomaly:.2f}"))
        
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0],
                           mode='markers', marker=dict(size=7, color='blue'), name='Earth'))

fig.update_layout(scene=dict(aspectmode='data'))
fig.show()


In [24]:
# arc creation

def arc_creation(N):
    A = []
    for i in range(len(N)):
        for j in range(len(N)):
            if i != j and N[i].id != 'E' and N[j].id != 'E':
                node_i = N[i]
                node_j = N[j]
                delta_theta = (angle_difference(round(node_i.true_anomaly.value, 2), round(node_j.true_anomaly.value, 2))) * u.deg

                if node_i.orbit == node_j.orbit:
                    
                    if round(360 * N[i].orbit.separation_sec / N[i].orbit.period.value, 2) * u.deg == delta_theta :
                        # Orbital maneuver
                        P = node_i.orbit.period
                        #tau_ij = ((delta_theta * np.pi) / 180) / (360 * u.deg) * (P / step_time)
                        tau_ij = round((P / node_i.orbit.nb_noeuds).value)
                        phi_ij = 0
                        time_period_needed = math.ceil(tau_ij / step_time)
                        psi_ij = round(phi_ij / time_period_needed, 2)
                        maneuver_type = "Orbital maneuver"
                        A.append(Arc(node_i, node_j, tau_ij, time_period_needed, phi_ij, psi_ij, maneuver_type))
                        
                        
                    else:
                        # Phasing maneuver
                        #delta_theta = round(delta_theta.value) * np.pi / 180   # convert in rad
                        if (N[j].true_anomaly - N[i].true_anomaly) < 0:
                            delta_phi = (360 + (N[j].true_anomaly).value - (N[i].true_anomaly).value) * np.pi / 180
                        elif (N[j].true_anomaly - N[i].true_anomaly) >= 0:
                            delta_phi = ((N[j].true_anomaly).value - (N[i].true_anomaly).value) * np.pi / 180
                        
                        #v = (np.sqrt((mu_earth) / (node_i.orbit.a))).value       # mean motion of the satellite in its circular orbit in km/s
                        #n_rad = (v*3600) / (3.6 * (N[i].orbit.a).value * 10**3)  # mean motion of the satellite in its circular orbit in rad/s
                        n_rad = (node_i.orbit.n).value 
                        k = 1
                        
                        T_phasing_ellipse = ((2 * k * np.pi) + delta_phi) / (k * n_rad)
                        #print('\nT: ', T_phasing_ellipse, '\n')
                        a_phasing_ellipse = ((mu_earth * T_phasing_ellipse**2) / (4 * np.pi**2)) ** (1/3)
                        #print('\na: ', a_phasing_ellipse, '\n')
    
                        delta_V = 2 * abs(np.sqrt(((2 * mu_earth) / (N[i].orbit.a).value) - (mu_earth / a_phasing_ellipse)) - np.sqrt(mu_earth / (N[i].orbit.a).value))
                        #print('\ndeltaV: ', delta_V, '\n')
                        tau_ij = round(((T_phasing_ellipse * delta_phi) / (2 * np.pi)) + (k * T_phasing_ellipse))
                        
                        phi_ij = round(delta_V, 2)

                        time_period_needed = math.ceil(tau_ij / step_time)
                        psi_ij = round(delta_V / time_period_needed, 2)
                        maneuver_type = "Phasing maneuver"
                        A.append(Arc(node_i, node_j, tau_ij, time_period_needed, phi_ij, psi_ij, maneuver_type))
                        
                        
                elif (node_i.orbit.inc == node_j.orbit.inc) and (node_i.orbit.a != node_j.orbit.a) and (abs(delta_theta) == 180 * u.deg):
                    # Hohmann transfer
                    # Orbital parameter of the starting orbit
                    V1 = (N[i].orbit.v[1]).value
                    
                    # Orbital parameter of the tranfer orbit
                    r_p = (N[i].orbit.r_p).value
                    r_a = (N[j].orbit.r_a).value
                    h2 = np.sqrt(2 * mu_earth) * np.sqrt((r_p * r_a) / (r_p + r_a))
                    V2 = h2 / r_p
                    
                    # Orbital parameter of the final orbit
                    V3 = (N[j].orbit.v[1]).value
                    
                    # delta V for the maneuver
                    deltaV1 = V2 - V1
                    deltaV2 = V3 - V2
                    deltaV = abs(deltaV1) + abs(deltaV2)
                    
                    tau_ij = round((2 * np.pi * (r_a**(3/2))) / (2 * np.sqrt(mu_earth)))
                    phi_ij = round(deltaV, 2)

                    time_period_needed = math.ceil(tau_ij / step_time)
                    psi_ij = round(deltaV / time_period_needed, 2)
                    maneuver_type = "Hohmann transfer"
                    A.append(Arc(node_i, node_j, tau_ij, time_period_needed, phi_ij, psi_ij, maneuver_type))
                    
                
                elif (node_i.orbit.inc != node_j.orbit.inc) and (node_i.orbit.a != node_j.orbit.a) and (abs(delta_theta) == 180 * u.deg):
                    # Hohmann transfer with inclination change
                    r1 = (node_i.orbit.a).value
                    r2 = (node_j.orbit.a).value
                    inc1 = node_i.orbit.inc.value
                    inc2 = node_j.orbit.inc.value
                    delta_i = abs(inc2 - inc1)
                    
                    
                    # Calculate the velocity at the starting orbit
                    vi1 = np.sqrt(mu_earth / r1)
                    
                    # Calculate the angular momentum at the destination orbit
                    hj = np.sqrt(2 * mu_earth) * np.sqrt((r1 * r2) / (r1 + r2))

                    # Intermediate calculations
                    vi2 = hj / r1
                    vj2 = hj / r2
                    vj3 = np.sqrt(mu_earth / r2)
                    
                    # Change in inclination at the destination
                    delta_vj = np.sqrt(vj2**2 + vj3**2 - (2 * vj2 * vj3 * np.cos(delta_i)))
                    delta_vi = vi2 - vi1
                    delta_V_destination = delta_vi + delta_vj
                    
                    # Change in inclination at the origin
                    delta_vi_origin = np.sqrt(vi1**2 + vi2**2 - (2 * vi2 * vi1 * np.cos(delta_i)))
                    delta_vj_origin = vj3 - vj2
                    delta_V_origin = delta_vi_origin + delta_vj_origin
                    
                    # Select the smaller delta V
                    #print(delta_V_origin," ",delta_V_destination)
                    if delta_V_origin < delta_V_destination and delta_V_origin > 0:
                        phi_ij = round(delta_V_origin, 2)
                    else:
                        phi_ij = round(delta_V_destination, 2)
                    
                    tau_ij = round((2 * np.pi * (r2**(3/2))) / (2 * np.sqrt(mu_earth)))

                    time_period_needed = math.ceil(tau_ij / step_time)
                    psi_ij = round(phi_ij / time_period_needed, 2)
                    maneuver_type = "Hohmann transfer with inclination change"
                    A.append(Arc(node_i, node_j, tau_ij, time_period_needed, phi_ij, psi_ij, maneuver_type))
                
                
            elif i != j and N[j].id == 'E':
                # Arc to super sink node E
                tau_ij = round(step_time)
                phi_ij = 0
                psi_ij = 0
                time_period_needed = 1
                maneuver_type = "Super sink"
                A.append(Arc(N[i], N[j], tau_ij, time_period_needed, phi_ij, psi_ij, maneuver_type))
                    
                    
                
                
                #if round(tau_ij.value) <= time_horizon:
                    #A.append(Arc(node_i, node_j, tau_ij, phi_ij, psi_ij, maneuver_type))
                    
    return A



A = arc_creation(N)


In [25]:
# Arc print
printok = True

mean_time_phasing_leo_leo = []
mean_time_phasing_meo_meo = []
mean_time_phasing_geo_geo = []

mean_deltaV_phasing_leo_leo = []
mean_deltaV_phasing_meo_meo = []
mean_deltaV_phasing_geo_geo = []

mean_time_orbital_leo_leo = []
mean_time_orbital_meo_meo = []
mean_time_orbital_geo_geo = []

mean_deltaV_orbital_leo_leo = []
mean_deltaV_orbital_meo_meo = []
mean_deltaV_orbital_geo_geo = []

mean_time_HIC_leo_meo = []
mean_time_HIC_leo_geo = []
mean_time_HIC_meo_geo = []

mean_deltaV_HIC_leo_meo = []
mean_deltaV_HIC_leo_geo = []
mean_deltaV_HIC_meo_geo = []

mean_time_H_meo_geo = []
mean_deltaV_H_meo_geo = []

mean_time_HIC_meo_leo = []
mean_time_HIC_geo_leo = []
mean_time_HIC_geo_meo = []

mean_deltaV_HIC_meo_leo = []
mean_deltaV_HIC_geo_leo = []
mean_deltaV_HIC_geo_meo = []

mean_time_H_geo_meo = []
mean_deltaV_H_geo_meo = []


maneuver_counts = {
    "Orbital maneuver": 0,
    "Phasing maneuver": 0,
    "Hohmann transfer": 0,
    "Hohmann transfer with inclination change": 0,
    "Super sink": 0}


for arc in A:
    if printok:
        print(f"Arc from Node {arc.node_i.id} to Node {arc.node_j.id}: tau_ij = {arc.tau_ij}, phi_ij (DeltaV) = {arc.phi_ij}, psi_ij = {arc.psi_ij}, {arc.maneuver_type}")
        
    maneuver_counts[arc.maneuver_type] += 1

    if arc.node_i.orbit.ttype == "Leo" and arc.node_j.orbit.ttype == "Leo":
        if arc.maneuver_type == "Phasing maneuver":
            mean_deltaV_phasing_leo_leo.append(arc.phi_ij)
            mean_time_phasing_leo_leo.append(arc.tau_ij)
        elif arc.maneuver_type == "Orbital maneuver":
            mean_deltaV_orbital_leo_leo.append(arc.phi_ij)
            mean_time_orbital_leo_leo.append(arc.tau_ij)

    if arc.node_i.orbit.ttype == "Meo" and arc.node_j.orbit.ttype == "Meo":
        if arc.maneuver_type == "Phasing maneuver":
            mean_deltaV_phasing_meo_meo.append(arc.phi_ij)
            mean_time_phasing_meo_meo.append(arc.tau_ij)
        elif arc.maneuver_type == "Orbital maneuver":
            mean_deltaV_orbital_meo_meo.append(arc.phi_ij)
            mean_time_orbital_meo_meo.append(arc.tau_ij)

    if arc.node_i.orbit.ttype == "Geo" and arc.node_j.orbit.ttype == "Geo":
        if arc.maneuver_type == "Phasing maneuver":
            mean_deltaV_phasing_geo_geo.append(arc.phi_ij)
            mean_time_phasing_geo_geo.append(arc.tau_ij)
        elif arc.maneuver_type == "Orbital maneuver":
            mean_deltaV_orbital_geo_geo.append(arc.phi_ij)
            mean_time_orbital_geo_geo.append(arc.tau_ij)

    
    if arc.node_i.orbit.ttype == "Leo" and arc.node_j.orbit.ttype == "Meo":
        if arc.maneuver_type == "Hohmann transfer with inclination change":
            mean_deltaV_HIC_leo_meo.append(arc.phi_ij)
            mean_time_HIC_leo_meo.append(arc.tau_ij)

    if arc.node_i.orbit.ttype == "Leo" and arc.node_j.orbit.ttype == "Geo":
        if arc.maneuver_type == "Hohmann transfer with inclination change":
            mean_deltaV_HIC_leo_geo.append(arc.phi_ij)
            mean_time_HIC_leo_geo.append(arc.tau_ij)

    if arc.node_i.orbit.ttype == "Meo" and arc.node_j.orbit.ttype == "Geo":
        if arc.maneuver_type == "Hohmann transfer with inclination change":
            mean_deltaV_HIC_meo_geo.append(arc.phi_ij)
            mean_time_HIC_meo_geo.append(arc.tau_ij)
        if arc.maneuver_type == "Hohmann transfer":
            mean_deltaV_H_meo_geo.append(arc.phi_ij)
            mean_time_H_meo_geo.append(arc.tau_ij)

    
    if arc.node_i.orbit.ttype == "Meo" and arc.node_j.orbit.ttype == "Leo":
        if arc.maneuver_type == "Hohmann transfer with inclination change":
            mean_deltaV_HIC_meo_leo.append(arc.phi_ij)
            mean_time_HIC_meo_leo.append(arc.tau_ij)

    if arc.node_i.orbit.ttype == "Geo" and arc.node_j.orbit.ttype == "Leo":
        if arc.maneuver_type == "Hohmann transfer with inclination change":
            mean_deltaV_HIC_geo_leo.append(arc.phi_ij)
            mean_time_HIC_geo_leo.append(arc.tau_ij)

    if arc.node_i.orbit.ttype == "Geo" and arc.node_j.orbit.ttype == "Meo":
        if arc.maneuver_type == "Hohmann transfer with inclination change":
            mean_deltaV_HIC_geo_meo.append(arc.phi_ij)
            mean_time_HIC_geo_meo.append(arc.tau_ij)
        if arc.maneuver_type == "Hohmann transfer":
            mean_deltaV_H_geo_meo.append(arc.phi_ij)
            mean_time_H_geo_meo.append(arc.tau_ij)
    
    
    
    
print("\nMean time phasing leo: ", round(np.mean(mean_time_phasing_leo_leo) / 3600, 2)*u.h)
print("Mean time phasing meo: ", round(np.mean(mean_time_phasing_meo_meo) / 3600, 2)*u.h)
print("Mean time phasing geo: ", round(np.mean(mean_time_phasing_geo_geo) / 3600, 2)*u.h)

print("Mean deltaV phasing leo: ", round(np.mean(mean_deltaV_phasing_leo_leo), 2))
print("Mean deltaV phasing meo: ", round(np.mean(mean_deltaV_phasing_meo_meo), 2))
print("Mean deltaV phasing geo: ", round(np.mean(mean_deltaV_phasing_geo_geo), 2))


print("\nMean time orbital leo: ", round(np.mean(mean_time_orbital_leo_leo) / 3600, 2)*u.h)
print("Mean time orbital meo: ", round(np.mean(mean_time_orbital_meo_meo) / 3600, 2)*u.h)
print("Mean time orbital geo: ", round(np.mean(mean_time_orbital_geo_geo) / 3600, 2)*u.h)

print("Mean deltaV orbital leo: ", round(np.mean(mean_deltaV_orbital_leo_leo), 2))
print("Mean deltaV orbital meo: ", round(np.mean(mean_deltaV_orbital_meo_meo), 2))
print("Mean deltaV orbital geo: ", round(np.mean(mean_deltaV_orbital_geo_geo), 2))


print("\n\n Ascending")
print("Mean time HIC leo - meo: ", round(np.mean(mean_time_HIC_leo_meo) / 3600, 2)*u.h)
print("Mean time HIC leo - geo: ", round(np.mean(mean_time_HIC_leo_geo) / 3600, 2)*u.h)
print("Mean time HIC meo - geo: ", round(np.mean(mean_time_HIC_meo_geo) / 3600, 2)*u.h)

print("Mean deltaV HIC leo - meo: ", round(np.mean(mean_deltaV_HIC_leo_meo), 2))
print("Mean deltaV HIC leo - geo: ", round(np.mean(mean_deltaV_HIC_leo_geo), 2))
print("Mean deltaV HIC meo - geo: ", round(np.mean(mean_deltaV_HIC_meo_geo), 2))

print("\nMean time H meo - geo: ", round(np.mean(mean_time_H_meo_geo) / 3600, 2)*u.h)
print("Mean deltaV H meo - geo: ", round(np.mean(mean_deltaV_H_meo_geo), 2))


print("\n\n Descending")
print("Mean time HIC meo - leo: ", round(np.mean(mean_time_HIC_meo_leo) / 3600, 2)*u.h)
print("Mean time HIC geo - leo: ", round(np.mean(mean_time_HIC_geo_leo) / 3600, 2)*u.h)
print("Mean time HIC geo - meo: ", round(np.mean(mean_time_HIC_geo_meo) / 3600, 2)*u.h)

print("Mean deltaV HIC meo - leo: ", round(np.mean(mean_deltaV_HIC_meo_leo), 2))
print("Mean deltaV HIC geo - leo: ", round(np.mean(mean_deltaV_HIC_geo_leo), 2))
print("Mean deltaV HIC geo - meo: ", round(np.mean(mean_deltaV_HIC_geo_meo), 2))

print("\nMean time H geo - meo: ", round(np.mean(mean_time_H_geo_meo) / 3600, 2)*u.h)
print("Mean deltaV H geo - meo: ", round(np.mean(mean_deltaV_H_geo_meo), 2))



print("\nManeuver Counts:")
tot = 0
for maneuver, count in maneuver_counts.items():
    tot += count
    print(f"{maneuver}: {count}")
print("total: ",tot)


Arc from Node 0 to Node 1: tau_ij = 1800, phi_ij (DeltaV) = 0, psi_ij = 0.0, Orbital maneuver
Arc from Node 0 to Node 2: tau_ij = 15000, phi_ij (DeltaV) = 2.09, psi_ij = 0.23, Phasing maneuver
Arc from Node 0 to Node 15: tau_ij = 21600, phi_ij (DeltaV) = 8.24, psi_ij = 0.69, Hohmann transfer with inclination change
Arc from Node 0 to Node 27: tau_ij = 21600, phi_ij (DeltaV) = 7.44, psi_ij = 0.62, Hohmann transfer with inclination change
Arc from Node 0 to Node 39: tau_ij = 21600, phi_ij (DeltaV) = 7.44, psi_ij = 0.62, Hohmann transfer with inclination change
Arc from Node 0 to Node 51: tau_ij = 21600, phi_ij (DeltaV) = 7.44, psi_ij = 0.62, Hohmann transfer with inclination change
Arc from Node 0 to Node 69: tau_ij = 43200, phi_ij (DeltaV) = 7.02, psi_ij = 0.29, Hohmann transfer with inclination change
Arc from Node 1 to Node 0: tau_ij = 15000, phi_ij (DeltaV) = 2.09, psi_ij = 0.23, Phasing maneuver
Arc from Node 1 to Node 2: tau_ij = 1800, phi_ij (DeltaV) = 0, psi_ij = 0.0, Orbital man

In [26]:
""" ---- MAx / Min phasing ---- """
print("\nMax time phasing leo: ", round(max(mean_time_phasing_leo_leo) / 3600, 2)*u.h)
print("Max time phasing meo: ", round(max(mean_time_phasing_meo_meo) / 3600, 2)*u.h)
print("Max time phasing geo: ", round(max(mean_time_phasing_geo_geo) / 3600, 2)*u.h)

print("Max deltaV phasing leo: ", round(max(mean_deltaV_phasing_leo_leo), 2))
print("Max deltaV phasing meo: ", round(max(mean_deltaV_phasing_meo_meo), 2))
print("Max deltaV phasing geo: ", round(max(mean_deltaV_phasing_geo_geo), 2))

print("\nMin time phasing leo: ", round(min(mean_time_phasing_leo_leo) / 3600, 2)*u.h)
print("Min time phasing meo: ", round(min(mean_time_phasing_meo_meo) / 3600, 2)*u.h)
print("Min time phasing geo: ", round(min(mean_time_phasing_geo_geo) / 3600, 2)*u.h)

print("Min deltaV phasing leo: ", round(min(mean_deltaV_phasing_leo_leo), 2))
print("Min deltaV phasing meo: ", round(min(mean_deltaV_phasing_meo_meo), 2))
print("Min deltaV phasing geo: ", round(min(mean_deltaV_phasing_geo_geo), 2))

print("\nLen phasing leo: ", len(mean_deltaV_phasing_leo_leo))
print("Len phasing meo: ", len(mean_deltaV_phasing_meo_meo))
print("Len phasing geo: ", len(mean_deltaV_phasing_geo_geo))



""" ---- MAx / Min orbital ---- """
print("\n\nMax time orbital leo: ", round(max(mean_time_orbital_leo_leo) / 3600, 2)*u.h)
print("Max time orbital meo: ", round(max(mean_time_orbital_meo_meo) / 3600, 2)*u.h)
print("Max time orbital geo: ", round(max(mean_time_orbital_geo_geo) / 3600, 2)*u.h)

print("Max deltaV orbital leo: ", round(max(mean_deltaV_orbital_leo_leo), 2))
print("Max deltaV orbital meo: ", round(max(mean_deltaV_orbital_meo_meo), 2))
print("Max deltaV orbital geo: ", round(max(mean_deltaV_orbital_geo_geo), 2))

print("\nMin time orbital leo: ", round(min(mean_time_orbital_leo_leo) / 3600, 2)*u.h)
print("Min time orbital meo: ", round(min(mean_time_orbital_meo_meo) / 3600, 2)*u.h)
print("Min time orbital geo: ", round(min(mean_time_orbital_geo_geo) / 3600, 2)*u.h)

print("Min deltaV orbital leo: ", round(min(mean_deltaV_orbital_leo_leo), 2))
print("Min deltaV orbital meo: ", round(min(mean_deltaV_orbital_meo_meo), 2))
print("Min deltaV orbital geo: ", round(min(mean_deltaV_orbital_geo_geo), 2))

print("\nLen orbital leo: ", len(mean_deltaV_orbital_leo_leo))
print("Len orbital meo: ", len(mean_deltaV_orbital_meo_meo))
print("Len orbital geo: ", len(mean_deltaV_orbital_geo_geo))



""" ---- MAx / Min Ascending ---- """
print("\n\n MAX Ascending")
print("Max time HIC leo - meo: ", round(max(mean_time_HIC_leo_meo) / 3600, 2)*u.h)
print("Max time HIC leo - geo: ", round(max(mean_time_HIC_leo_geo) / 3600, 2)*u.h)
print("Max time HIC meo - geo: ", round(max(mean_time_HIC_meo_geo) / 3600, 2)*u.h)

print("Max deltaV HIC leo - meo: ", round(max(mean_deltaV_HIC_leo_meo), 2))
print("Max deltaV HIC leo - geo: ", round(max(mean_deltaV_HIC_leo_geo), 2))
print("Max deltaV HIC meo - geo: ", round(max(mean_deltaV_HIC_meo_geo), 2))

print("\nMax time H meo - geo: ", round(max(mean_time_H_meo_geo) / 3600, 2)*u.h)
print("Max deltaV H meo - geo: ", round(max(mean_deltaV_H_meo_geo), 2))

print("\n MIN Ascending")
print("Min time HIC leo - meo: ", round(min(mean_time_HIC_leo_meo) / 3600, 2)*u.h)
print("Min time HIC leo - geo: ", round(min(mean_time_HIC_leo_geo) / 3600, 2)*u.h)
print("Min time HIC meo - geo: ", round(min(mean_time_HIC_meo_geo) / 3600, 2)*u.h)

print("Min deltaV HIC leo - meo: ", round(min(mean_deltaV_HIC_leo_meo), 2))
print("Min deltaV HIC leo - geo: ", round(min(mean_deltaV_HIC_leo_geo), 2))
print("Min deltaV HIC meo - geo: ", round(min(mean_deltaV_HIC_meo_geo), 2))

print("\nMin time H meo - geo: ", round(min(mean_time_H_meo_geo) / 3600, 2)*u.h)
print("Min deltaV H meo - geo: ", round(min(mean_deltaV_H_meo_geo), 2))

print("\nLen HIC leo - meo: ", len(mean_deltaV_HIC_leo_meo))
print("Len HIC leo - geo: ", len(mean_deltaV_HIC_leo_geo))
print("Len HIC meo - geo: ", len(mean_deltaV_HIC_meo_geo))
print("Len H meo - geo: ", len(mean_deltaV_H_meo_geo))


""" ---- MAx / Min Descending ---- """
print("\n\n Descending")
print("Max time HIC meo - leo: ", round(max(mean_time_HIC_meo_leo) / 3600, 2)*u.h)
print("Max time HIC geo - leo: ", round(max(mean_time_HIC_geo_leo) / 3600, 2)*u.h)
print("Max time HIC geo - meo: ", round(max(mean_time_HIC_geo_meo) / 3600, 2)*u.h)

print("Max deltaV HIC meo - leo: ", round(max(mean_deltaV_HIC_meo_leo), 2))
print("Max deltaV HIC geo - leo: ", round(max(mean_deltaV_HIC_geo_leo), 2))
print("Max deltaV HIC geo - meo: ", round(max(mean_deltaV_HIC_geo_meo), 2))

print("\nMax time H geo - meo: ", round(max(mean_time_H_geo_meo) / 3600, 2)*u.h)
print("Max deltaV H geo - meo: ", round(max(mean_deltaV_H_geo_meo), 2))

print("\n\n Descending")
print("Min time HIC meo - leo: ", round(min(mean_time_HIC_meo_leo) / 3600, 2)*u.h)
print("Min time HIC geo - leo: ", round(min(mean_time_HIC_geo_leo) / 3600, 2)*u.h)
print("Min time HIC geo - meo: ", round(min(mean_time_HIC_geo_meo) / 3600, 2)*u.h)

print("Min deltaV HIC meo - leo: ", round(min(mean_deltaV_HIC_meo_leo), 2))
print("Min deltaV HIC geo - leo: ", round(min(mean_deltaV_HIC_geo_leo), 2))
print("Min deltaV HIC geo - meo: ", round(min(mean_deltaV_HIC_geo_meo), 2))

print("\nMin time H geo - meo: ", round(min(mean_time_H_geo_meo) / 3600, 2)*u.h)
print("Min deltaV H geo - meo: ", round(min(mean_deltaV_H_geo_meo), 2))

print("\nLen HIC meo - leo: ", len(mean_deltaV_HIC_meo_leo))
print("Len HIC geo - leo: ", len(mean_deltaV_HIC_geo_leo))
print("Len HIC geo - meo: ", len(mean_deltaV_HIC_geo_meo))
print("Len H geo - meo: ", len(mean_deltaV_H_geo_meo))



Max time phasing leo:  4.17 h
Max time phasing meo:  44.08 h
Max time phasing geo:  92.04 h
Max deltaV phasing leo:  2.09
Max deltaV phasing meo:  1.26
Max deltaV phasing geo:  1.02

Min time phasing leo:  4.17 h
Min time phasing meo:  16.33 h
Min time phasing geo:  28.17 h
Min deltaV phasing leo:  2.09
Min deltaV phasing meo:  0.37
Min deltaV phasing geo:  0.16

Len phasing leo:  9
Len phasing meo:  480
Len phasing geo:  528


Max time orbital leo:  0.5 h
Max time orbital meo:  1.0 h
Max time orbital geo:  1.0 h
Max deltaV orbital leo:  0
Max deltaV orbital meo:  0
Max deltaV orbital geo:  0

Min time orbital leo:  0.5 h
Min time orbital meo:  1.0 h
Min time orbital geo:  1.0 h
Min deltaV orbital leo:  0
Min deltaV orbital meo:  0
Min deltaV orbital geo:  0

Len orbital leo:  9
Len orbital meo:  48
Len orbital geo:  24


 MAX Ascending
Max time HIC leo - meo:  6.0 h
Max time HIC leo - geo:  12.0 h
Max time HIC meo - geo:  12.0 h
Max deltaV HIC leo - meo:  8.24
Max deltaV HIC leo - ge

In [27]:
# Function to create tasks

def create_tasks(num_tasks_per_orbit, O, starting_node_task = []):
    B = []
    task_id = 0
    count = 0
    for orbit in O:
        orbit_nodes = orbit.noeuds
        for _ in range(num_tasks_per_orbit):
            task = Task(task_id, 1, orbit)
            
            if starting_node_task == []:                            # We check if the "starting_node_task" is empty
                task.starting_node = random.choice(orbit_nodes)     # If this is the case we choose the starting nodes randomly
            else:
                task.starting_node = orbit_nodes[starting_node_task[count]]    # Otherwise, we choose the starting nodes according to the list "starting_node_task"       
                
            B.append(task)
            task_id += 1
            count += 1
            
    return B


# Sub Task Creation Algorithm
def create_sub_tasks(T, N, A, B, alpha):
    for v in B:
        Bv = []
        current_node = v.starting_node
        current_time = 0   # Initialize with units
        Bv.append(SubTask(current_node, current_time, v))
        
        while current_time < time_horizon_steptime :
            
            # Find the next node on the orbit in the rotational direction
            next_node = None
            for arc in A:
                if arc.node_i == current_node and arc.maneuver_type == "Orbital maneuver":
                    next_node = arc.node_j
                    current_time += arc.tau_ij_steptime
                    break
                    
            current_node = next_node

            # Add sub task to Bv associated with the (current_node, current_time)
            Bv.append(SubTask(current_node, current_time, v))
            
            if next_node is None:
                print("\nNext node not found\n")
                break  # Exit the loop if no next node is found
        
        v.sub_tasks = Bv
    
    return B



# Parameters
starting_node_task = [0, 1, 2,  1, 2, 0,  2, 0, 1,  0, 4, 8,  2, 6, 10,  1, 5, 9,  3, 7, 11,  0, 8, 16]
#starting_node_task_test = [0, 2, 4,  1, 3, 5,  0, 2, 4,  0, 8, 16,  2, 10, 18,  4, 12, 20,  3, 11, 19,  0, 16, 32]
#starting_node_task_test2 = [0, 1, 2,  1, 2, 0,  2, 0, 1,  0, 4, 8,  2, 6, 10,  1, 5, 9,  3, 7, 11,  0, 8, 16]
num_tasks_per_orbit = 3


# Create tasks
B = create_tasks(num_tasks_per_orbit, O, starting_node_task)

# Run the algorithm
T = time_horizon
B_with_sub_tasks = create_sub_tasks(T, N, A, B, 0.00001)

# Print the results
for task in B_with_sub_tasks:
    print(f"Task {task.id} (Starting Node: {task.starting_node.id}, Orbit: {O.index(task.starting_orbit)}):")
    print(f"  Number of sub-tasks: {len(task.sub_tasks)}")
    for sub_task in task.sub_tasks:
        print(f"    Node: {sub_task.node.id}, Time: {sub_task.time}, Completed: {sub_task.completed}")

Task 0 (Starting Node: 0, Orbit: 0):
  Number of sub-tasks: 1441
    Node: 0, Time: 0, Completed: 0
    Node: 1, Time: 1, Completed: 0
    Node: 2, Time: 2, Completed: 0
    Node: 0, Time: 3, Completed: 0
    Node: 1, Time: 4, Completed: 0
    Node: 2, Time: 5, Completed: 0
    Node: 0, Time: 6, Completed: 0
    Node: 1, Time: 7, Completed: 0
    Node: 2, Time: 8, Completed: 0
    Node: 0, Time: 9, Completed: 0
    Node: 1, Time: 10, Completed: 0
    Node: 2, Time: 11, Completed: 0
    Node: 0, Time: 12, Completed: 0
    Node: 1, Time: 13, Completed: 0
    Node: 2, Time: 14, Completed: 0
    Node: 0, Time: 15, Completed: 0
    Node: 1, Time: 16, Completed: 0
    Node: 2, Time: 17, Completed: 0
    Node: 0, Time: 18, Completed: 0
    Node: 1, Time: 19, Completed: 0
    Node: 2, Time: 20, Completed: 0
    Node: 0, Time: 21, Completed: 0
    Node: 1, Time: 22, Completed: 0
    Node: 2, Time: 23, Completed: 0
    Node: 0, Time: 24, Completed: 0
    Node: 1, Time: 25, Completed: 0
    Node:

In [28]:
# Create refueling Arc 

num_depot_per_orbit = 1

def create_refueling_depot(num_depot_per_orbit, O, starting_node_r = []):
    R = []
    depot_id = 0
    count = 0
    for orbit in O:
        orbit_nodes = orbit.noeuds
        for _ in range(num_depot_per_orbit):
            depot = refueling_depot(depot_id, orbit)

            if starting_node_r == []:                               # We check if the "starting_node_r" is empty
                depot.starting_node = random.choice(orbit_nodes)    # If this is the case we choose the starting nodes randomly
            else:
                depot.starting_node = orbit_nodes[starting_node_r[count]]   # Otherwise, we choose the starting nodes according to the list "starting_node_r"       
                
            R.append(depot)
            depot_id += 1
            count += 1
        
    return R




def refueling_arc(T, N, A, R):
    
    AR = {t: [] for t in T}
    
    for depot in R:
        current_node = depot.starting_node
        next_node = None
        for arc in A:
            if arc.node_i == current_node and arc.maneuver_type == "Orbital maneuver":
                next_node = arc.node_j
                break
        
        current_time = 0 

        while current_time <= T[-1] :
            AR[current_time].append((current_node, next_node))
            
            current_node = next_node
            for arc in A:
                if arc.node_i == current_node and arc.maneuver_type == "Orbital maneuver":
                    next_node = arc.node_j
                    current_time += arc.tau_ij_steptime
                    break
            
            if next_node is None:
                print("\nNext node not found\n")
                break  # Exit the loop if no next node is found
                
    return AR



T_list = [t for t in range(time_horizon_steptime + 1)]


num_depot_per_orbit = 1
starting_node_r = [1, 0, 1, 4, 5, 8, 2, 12]
#starting_node_r_test = [0, 0, 1]

R = create_refueling_depot(num_depot_per_orbit, O, starting_node_r)

AR = refueling_arc(T_list, N, A, R)

for time in AR.keys():
    print ("time : ", time)
    for re in range(len(AR[time])):
        print(AR[time][re][0].id, "->", AR[time][re][1].id)



time :  0
1 -> 2
3 -> 4
7 -> 8
13 -> 14
26 -> 27
41 -> 42
47 -> 48
69 -> 70
time :  1
2 -> 0
4 -> 5
8 -> 6
time :  2
0 -> 1
5 -> 3
6 -> 7
14 -> 15
27 -> 28
42 -> 43
48 -> 49
70 -> 71
time :  3
1 -> 2
3 -> 4
7 -> 8
time :  4
2 -> 0
4 -> 5
8 -> 6
15 -> 16
28 -> 29
43 -> 44
49 -> 50
71 -> 72
time :  5
0 -> 1
5 -> 3
6 -> 7
time :  6
1 -> 2
3 -> 4
7 -> 8
16 -> 17
29 -> 30
44 -> 33
50 -> 51
72 -> 73
time :  7
2 -> 0
4 -> 5
8 -> 6
time :  8
0 -> 1
5 -> 3
6 -> 7
17 -> 18
30 -> 31
33 -> 34
51 -> 52
73 -> 74
time :  9
1 -> 2
3 -> 4
7 -> 8
time :  10
2 -> 0
4 -> 5
8 -> 6
18 -> 19
31 -> 32
34 -> 35
52 -> 53
74 -> 75
time :  11
0 -> 1
5 -> 3
6 -> 7
time :  12
1 -> 2
3 -> 4
7 -> 8
19 -> 20
32 -> 21
35 -> 36
53 -> 54
75 -> 76
time :  13
2 -> 0
4 -> 5
8 -> 6
time :  14
0 -> 1
5 -> 3
6 -> 7
20 -> 9
21 -> 22
36 -> 37
54 -> 55
76 -> 77
time :  15
1 -> 2
3 -> 4
7 -> 8
time :  16
2 -> 0
4 -> 5
8 -> 6
9 -> 10
22 -> 23
37 -> 38
55 -> 56
77 -> 78
time :  17
0 -> 1
5 -> 3
6 -> 7
time :  18
1 -> 2
3 -> 4
7 -> 8

In [29]:
r1 = Robot_serviser(ID = 1, max_fuel = 30, current_orbit = orbit1, current_node = N[0], R_Fd = 30)
D = {
    "d1" : r1
}


In [30]:
# Function to set the maximum amount of fuel that servicer 𝑑 takes on while traversing arc (𝑖, 𝑗)

def f_dij_function(D, AR, A):
    # Initialisation du dictionnaire de dictionnaire f_dij
    f_dij = {}

    # Itération sur les robots dans D
    for robot_key, robot in D.items():
        # Initialisation du sous-dictionnaire pour chaque robot
        f_dij[robot_key] = {}

        # On stocke les arcs uniques dans un ensemble (set)
        unique_arcs = set()

        # Itération sur le dictionnaire AR pour collecter les arcs
        for time, arcs in AR.items():
            for arc in arcs:
                # Création de la représentation des arcs sous forme de tuple (node_i, node_j)
                arc_tuple = (arc[0].id, arc[1].id)
                # Ajout de l'arc à l'ensemble (cela évite les doublons)
                unique_arcs.add(arc_tuple)

        # Transformation de l'ensemble unique_arcs en dictionnaire où chaque arc est une clé
        # Chaque arc correspond à une valeur initialisée à zéro (ou une autre valeur par défaut)
        for arc_R in unique_arcs:
            for arc in A:
                if arc.node_i.id == arc_R[0] and arc.node_j.id == arc_R[1]:
                    f_dij[robot_key][arc_R] = (arc.tau_ij_steptime / D[robot_key].R_Fd) * D[robot_key].max_fuel

    return f_dij


# Appel de la fonction
f_dij = f_dij_function(D, AR, A)

# Résultat du dictionnaire f_dij
print(f_dij)



{'d1': {(72, 73): 2.0, (67, 68): 2.0, (40, 41): 2.0, (32, 21): 2.0, (41, 42): 2.0, (73, 74): 2.0, (18, 19): 2.0, (5, 3): 1.0, (14, 15): 2.0, (9, 10): 2.0, (74, 75): 2.0, (15, 16): 2.0, (47, 48): 2.0, (42, 43): 2.0, (38, 39): 2.0, (48, 49): 2.0, (75, 76): 2.0, (16, 17): 2.0, (71, 72): 2.0, (44, 33): 2.0, (12, 13): 2.0, (22, 23): 2.0, (77, 78): 2.0, (49, 50): 2.0, (13, 14): 2.0, (45, 46): 2.0, (55, 56): 2.0, (50, 51): 2.0, (51, 52): 2.0, (46, 47): 2.0, (23, 24): 2.0, (78, 79): 2.0, (19, 20): 2.0, (24, 25): 2.0, (79, 80): 2.0, (52, 53): 2.0, (57, 58): 2.0, (53, 54): 2.0, (30, 31): 2.0, (25, 26): 2.0, (26, 27): 2.0, (21, 22): 2.0, (31, 32): 2.0, (8, 6): 1.0, (17, 18): 2.0, (27, 28): 2.0, (54, 55): 2.0, (64, 65): 2.0, (60, 61): 2.0, (1, 2): 1.0, (28, 29): 2.0, (56, 45): 2.0, (20, 9): 2.0, (80, 57): 2.0, (29, 30): 2.0, (61, 62): 2.0, (62, 63): 2.0, (3, 4): 1.0, (58, 59): 2.0, (35, 36): 2.0, (36, 37): 2.0, (68, 69): 2.0, (63, 64): 2.0, (4, 5): 1.0, (59, 60): 2.0, (69, 70): 2.0, (0, 1): 1.0, (

# Type of maintenance parameters

In [31]:
def get_random_initial_operating_time(H: int, h: int, n: int):
    max_value = H + h
    return np.random.randint(0, max_value + 1, size=n)

In [32]:
H = 1008   # failure threshold
h = 480    # time window to perform the preventive maintenance: H-h <= t < H

c_PM = 10    # cost of performing a preventive maintenance
c_CM = 25    # cost of performing a corrective maintenance
c_OP = 5    # opportunity cost

n = len(B)
#a0 = get_random_initial_operating_time(H, h, n)   # generate the operating time randomly
a0 = np.array((769, 978, 489,  591, 588, 394,  488, 812, 315,  666, 692, 489,  782, 589, 674,  467, 589, 836,  763, 188, 293,  890, 183, 328))                        # or we can set it up manualy 


tf = np.maximum(H - a0, 1)            # in how many timestep the satellite i will fail

pm_min = H - a0 - h                   # in how many timestep we can perform a PM


In [33]:
for task in B:
    print(task.id)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


# Data exportation

In [34]:
# ------------ Export the data in CSV file ------------ #


starting_node_task_str = ','.join(map(str, starting_node_task))
starting_node_r_str = ','.join(map(str, starting_node_r))
study_case_name = "30d_study_case"
output_directory = f'/Users/nathanclaret/Desktop/thesis/code/data_studycase/' + study_case_name
os.makedirs(output_directory, exist_ok=True)

# ------------ define all the parameters for the current case study ------------ #

param_lines = [
    "===== STUDY‑CASE PARAMETERS =====",
    f"Max fuel capacity (Fd)                   : {r1.max_fuel}",
    f"Nb of time period to refuel to Fd (RFd)  : {r1.R_Fd}",
    f"Starting orbit / node                    : {r1.current_orbit.ttype}  (node n{r1.current_node.id})",
    f"Time horizon (s)                         : {time_horizon} = {time_horizon/24/60/60} d",
    f"Task starting nodes (starting_t)         : {starting_node_task_str or 'random'}",
    f"Depot starting nodes (starting_r)        : {starting_node_r_str or 'random'}",
    f"Initial operating times OT (a0)          : {', '.join(map(str, a0))}",
    "",
    "----- Maintenance model constants -----",
    f"Failure threshold  H             : {H}",
    f"Preventive window   h            : {h}",
    f"Cost c_PM (preventive)           : {c_PM}",
    f"Cost c_CM (corrective)           : {c_CM}",
    f"Cost c_OP (opportunity)          : {c_OP}",
    "",
    "----- Simulation parameters -----",
    f"Step time (s)                    : {step_time}",
]

# Écriture du fichier texte dans le même dossier
with open(os.path.join(output_directory, 'parameters.txt'), 'w', encoding='utf-8') as paramfile:
    paramfile.write('\n'.join(param_lines))

# Export nodes
with open(os.path.join(output_directory, 'nodes.csv'), 'w', newline='') as csvfile:
    fieldnames = ['Node', 'ID']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for node in N:
        if node.id == "E":
            writer.writerow({'Node': 'E', 'ID': len(N)-1})
        else:
            writer.writerow({'Node': f'n{node.id}', 'ID': node.id})


# Export tasks
with open(os.path.join(output_directory, 'tasks.csv'), 'w', newline='') as csvfile:
    fieldnames = ['Task', 'Weight']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for task in B:
        writer.writerow({'Task': f'v{task.id + 1}', 'Weight': task.weight})


# Export subtasks
with open(os.path.join(output_directory, 'subtasks.csv'), 'w', newline='') as csvfile:
    fieldnames = ['Task', 'SubTask', 'Node', 'Time']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for task in B_with_sub_tasks:
        subtask_id = 1
        for sub_task in task.sub_tasks:
            writer.writerow({'Task': f'v{task.id + 1}', 'SubTask': f'k{subtask_id}', 'Node': f'n{sub_task.node.id}', 'Time': sub_task.time})
            subtask_id += 1


# Export arcs
with open(os.path.join(output_directory, 'arcs.csv'), 'w', newline='') as csvfile:
    fieldnames = ['FromNode', 'ToNode', 'Tau', 'Phi', 'Psi']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for arc in A:
        if arc.node_j.id != 'E':
            writer.writerow({'FromNode': f'n{arc.node_i.id}', 'ToNode': f'n{arc.node_j.id}', 'Tau': arc.tau_ij_steptime, 'Phi': arc.phi_ij, 'Psi': arc.psi_ij})
        else:
            writer.writerow({'FromNode': f'n{arc.node_i.id}', 'ToNode': arc.node_j.id, 'Tau': arc.tau_ij_steptime, 'Phi': arc.phi_ij, 'Psi': arc.psi_ij})


# Export robot servicer data
with open(os.path.join(output_directory, 'robots.csv'), 'w', newline='') as csvfile:
    fieldnames = ['RobotID', 'StartingNode', 'MaxFuel', 'R_Fd']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({'RobotID': f'd{r1.id}', 'StartingNode': f'n{r1.current_node.id}', 'MaxFuel': r1.max_fuel, 'R_Fd': r1.R_Fd})


# Export refueling arcs
with open(os.path.join(output_directory, 'refueling_arcs.csv'), 'w', newline='') as csvfile:
    fieldnames = ['Time', 'Arcs']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    # Loop through the AR dictionary and convert it into the required format
    for time, arcs in AR.items():
        # Create a list of "nX => nY" strings from the arcs for this time step
        arc_strings = [f"n{arc[0].id} => n{arc[1].id}" for arc in arcs]

        # Join all arc strings into a single string for the CSV
        arcs_formatted = ', '.join(arc_strings)

        # Write the row to the CSV
        writer.writerow({'Time': time, 'Arcs': arcs_formatted})


# Export f_dij
with open(os.path.join(output_directory, 'f_dij.csv'), 'w', newline='') as csvfile:
    fieldnames = ['Robot', 'Arc', 'Value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    # Parcourir chaque robot et ses arcs dans f_dij
    for robot_key, arcs_dict in f_dij.items():
        for arc, value in arcs_dict.items():
            # Convertir l'arc en format "nX => nY"
            arc_string = f"n{arc[0]} => n{arc[1]}"
            # Écrire une ligne avec le robot, l'arc et la valeur
            writer.writerow({'Robot': robot_key, 'Arc': arc_string, 'Value': value})


# Export opereting time and time window

with open(os.path.join(output_directory, 'maintenance_parameters.csv'), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # ----- bloc 1 : métadonnées -----
    writer.writerow(['Parameter', 'Value'])
    writer.writerow(['H', H])
    writer.writerow(['h', h])
    writer.writerow(['c_PM', c_PM])
    writer.writerow(['c_CM', c_CM])
    writer.writerow(['c_OP', c_OP])

    # ligne vide pour séparer
    writer.writerow([])

    # ----- bloc 2 : fenêtres de maintenance -----
    writer.writerow(['Task', 'OperatingTime_a0'])
    for i, task in enumerate(B):
        writer.writerow([f'v{task.id + 1}', int(a0[i])])


